In [111]:
# The code was removed by Watson Studio for sharing.

In [112]:
!pip install --user folium

In [113]:
### Importing libraries 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import bs4 as bs

import urllib.request

import requests

In [114]:
# The code was removed by Watson Studio for sharing.

Your credentails: hidden


In [116]:
### Commented out, data has been saved as CSV and called below t avoids repeated API calls

"""

source = urllib.request.urlopen('http://maps.latimes.com/neighborhoods/neighborhood/list/').read()
soup=bs.BeautifulSoup(source, 'lxml')

table = soup.find('table')

table_rows = table.find_all('tr')

ls = []
for tr in table_rows:
    td = tr.find_all('a')
    row = [tr.text.strip() for tr in td]
    ls.append(row)
    
LA_neighs = pd.DataFrame(ls, columns = ['Neighborhood','Region'])

LA_neighs=LA_neighs.drop(LA_neighs.index[0])

LA_neighs.tail()

"""

"\n\nsource = urllib.request.urlopen('http://maps.latimes.com/neighborhoods/neighborhood/list/').read()\nsoup=bs.BeautifulSoup(source, 'lxml')\n\ntable = soup.find('table')\n\ntable_rows = table.find_all('tr')\n\nls = []\nfor tr in table_rows:\n    td = tr.find_all('a')\n    row = [tr.text.strip() for tr in td]\n    ls.append(row)\n    \nLA_neighs = pd.DataFrame(ls, columns = ['Neighborhood','Region'])\n\nLA_neighs=LA_neighs.drop(LA_neighs.index[0])\n\nLA_neighs.tail()\n\n"

In [117]:
### Commented out, data has been saved as CSV and called below t avoids repeated API calls

"""

latitude = []
longitude = []

for name in LA_neighs['Neighborhood']:
    try:
        address = str(name)+", Los Angeles, California"
        geolocator = Nominatim(user_agent="LA_explorer")
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(None)
        longitude.append(None)

LA_neighs['Latitude'] = latitude
LA_neighs['Longitude'] = longitude

"""

'\n\nlatitude = []\nlongitude = []\n\nfor name in LA_neighs[\'Neighborhood\']:\n    try:\n        address = str(name)+", Los Angeles, California"\n        geolocator = Nominatim(user_agent="LA_explorer")\n        location = geolocator.geocode(address)\n        latitude.append(location.latitude)\n        longitude.append(location.longitude)\n    except:\n        latitude.append(None)\n        longitude.append(None)\n\nLA_neighs[\'Latitude\'] = latitude\nLA_neighs[\'Longitude\'] = longitude\n\n'

In [118]:
### Commented out, data has been saved as CSV and called below t avoids repeated API calls

"""

LA_neighs.dropna(inplace = True)

"""

'\n\nLA_neighs.dropna(inplace = True)\n\n'

In [119]:
### Commented out, data has been saved as CSV and called below t avoids repeated API calls

"""

LA_neighs[0:50]

"""

'\n\nLA_neighs[0:50]\n\n'

In [120]:
### Commented out, data has been saved as CSV and called below t avoids repeated API calls

"""

### Saving data to csv
project.save_data(data=LA_neighs.to_csv(index=False),file_name='LA_neighborhoods_v0.2.csv',overwrite=True)

"""

"\n\n### Saving data to csv\nproject.save_data(data=LA_neighs.to_csv(index=False),file_name='LA_neighborhoods_v0.2.csv',overwrite=True)\n\n"

In [121]:
#Getting csv file and loading as pandas datafarme
LA_neighs=pd.read_csv(project.get_file('LA_neighborhoods_v0.2.csv'))

LA_neighs.head()

,Neighborhood,Region,Latitude,Longitude
0,Acton,Antelope Valley,34.480742,-118.186838
1,Adams-Normandie,South L.A.,34.018609,-118.287348
2,Agoura Hills,Santa Monica Mountains,34.147910,-118.765704
3,Agua Dulce,Northwest County,34.496382,-118.325635
4,Alhambra,San Gabriel Valley,34.093042,-118.127060


In [122]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="LA_explorer")
location = geolocator.geocode(address)
LAlatitude = location.latitude
LAlongitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(LAlatitude, LAlongitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [123]:
# create map of LA using latitude and longitude values
map_LA = folium.Map(location=[LAlatitude, LAlongitude], zoom_start=11, width=800, height=600)

counter1=0

# add markers to map
for lat, lng, borough, neighborhood in zip(LA_neighs['Latitude'], LA_neighs['Longitude'], LA_neighs['Neighborhood'], LA_neighs['Region']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  
    counter1+=1
map_LA

In [124]:
LA_neighs.shape

(251, 4)

In [127]:
def getNearbyVenues(names, latitudes, longitudes, categoryIds, radius, limit):
    counter2 = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        counter2 += 1
        print(name+str(counter2))
            
        LIMIT = limit # limit of number of venues returned by Foursquare API
        radius = radius # define radius
        # create the API request URL
        categoryId = categoryIds
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            categoryId,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LA_venues = getNearbyVenues(LA_neighs['Neighborhood'], LA_neighs['Latitude'], LA_neighs['Longitude'], categoryIds = '4d4b7104d754a06370d81259', radius = 500, limit = 100)

Acton1
Adams-Normandie2
Agoura Hills3
Agua Dulce4
Alhambra5
Alondra Park6
Altadena7
Angeles Crest8
Arcadia9
Arleta10
Arlington Heights11
Artesia12
Athens13
Atwater Village14
Avalon15
Avocado Heights16
Azusa17
Baldwin Hills/Crenshaw18
Baldwin Park19
Bel-Air20
Bell21
Bellflower22
Bell Gardens23
Beverly Grove24
Beverly Hills25
Beverlywood26
Boyle Heights27
Bradbury28
Brentwood29
Burbank30
Calabasas31
Canoga Park32
Carson33
Carthay34
Castaic35
Central-Alameda36
Century City37
Cerritos38
Charter Oak39
Chatsworth40
Chatsworth Reservoir41
Cheviot Hills42
Chinatown43
Citrus44
Claremont45
Commerce46
Compton47
Covina48
Cudahy49
Culver City50
Cypress Park51
Del Aire52
Del Rey53
Desert View Highlands54
Diamond Bar55
Downey56
Downtown57
Duarte58
Eagle Rock59
East Compton60
East Hollywood61
East La Mirada62
East Los Angeles63
East Pasadena64
East San Gabriel65
Echo Park66
El Monte67
El Segundo68
El Sereno69
Elysian Park70
Elysian Valley71
Encino72
Exposition Park73
Fairfax74
Florence75
Florence-Fire

In [13]:
print(LA_venues.shape)

print('There are {} uniques categories.'.format(len(LA_venues['Venue Category'].unique())))

LA_venues.head()

(388, 7)
There are 21 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,34.018609,-118.287348,Metro Rail - Expo Park/USC Station (E),34.018237,-118.286094,Light Rail Station
1,Adams-Normandie,34.018609,-118.287348,Natural History Museum (NHM) Metro Bus 102/550,34.018072,-118.288654,Bus Stop
2,Adams-Normandie,34.018609,-118.287348,USC Tram Stop - Downey Way & Watt Way,34.020348,-118.287788,Bus Line
3,Adams-Normandie,34.018609,-118.287348,Metro Rail - Expo/Vermont Station (E),34.018241,-118.291541,Light Rail Station
4,Adams-Normandie,34.018609,-118.287348,USC Tram Stop - Parkside,34.019586,-118.290374,Bus Line


In [14]:
### Saving data to csv
project.save_data(data=LA_venues.to_csv(index=False),file_name='LA_venues_public_transport.csv',overwrite=True)

{'file_name': 'LA_venues_public_transport.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datasciencecapstone-donotdelete-pr-sodywh9vtibs6d',
 'asset_id': 'a226e2bb-e847-4446-a41e-084ba2d4be21'}

In [15]:

#Getting csv file and loading as pandas datafarme
LA_venues=pd.read_csv(project.get_file('LA_venues_public_transport.csv'))

LA_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
383,Woodland Hills,34.168436,-118.605838,Metro bus line 150,34.169470,-118.604288,Bus Line
384,Woodland Hills,34.168436,-118.605838,161 Metro Bus Stop,34.168665,-118.606203,Bus Line
385,Woodland Hills,34.168436,-118.605838,645 Metro Bus Stop,34.169521,-118.605736,Bus Line
386,Woodland Hills,34.168436,-118.605838,L.A. Metro Bus 244,34.168660,-118.604112,Bus Line
387,Woodland Hills,34.168436,-118.605838,Metro 161 W (Topanga/Ventura),34.168684,-118.610499,Bus Line


In [16]:
LAsum=LA_venues['Neighborhood'].unique()
LAsum.shape

(126,)

In [17]:
# create map of LA food venues using latitude and longitude values
map_LA_food = folium.Map(location=[LAlatitude, LAlongitude], zoom_start=11, width=1600, height=1200)

# add markers to map
for lat, lng, venue, venuecat in zip(LA_venues['Venue Latitude'], LA_venues['Venue Longitude'], LA_venues['Venue'], LA_venues['Venue Category']):
    label = '{}, {}'.format(venue, venuecat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA_food)  
    
map_LA_food

In [18]:
## Unique venues


n_unique = len(LA_venues['Venue Category'].unique())
print(f'There are {n_unique} unique venue categories in this dataframe')

LA_grouped=(LA_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False))
LA_grouped

There are 21 unique venue categories in this dataframe


Venue Category
Bus Line                    187
Bus Station                  77
Bus Stop                     49
Light Rail Station           16
Bike Shop                    13
Train Station                12
Bike Rental / Bike Share     11
Metro Station                 4
Parking                       2
Train                         2
General Travel                2
Platform                      2
Road                          2
Trail                         2
Outdoor Sculpture             1
Office                        1
Café                          1
Rental Car Location           1
Building                      1
Athletics & Sports            1
Arcade                        1
Name: Venue Category, dtype: int64

In [19]:
LA_venues.isnull().values.any()

False

In [20]:
## One-hot encoding

LA_venues_onehot = pd.get_dummies(LA_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
LA_venues_onehot['Neighborhood'] = LA_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [LA_venues_onehot.columns[-1]] + list(LA_venues_onehot.columns[:-1])
LA_venues_onehot = LA_venues_onehot[fixed_columns]

print(LA_venues_onehot.shape)
LA_venues_onehot.head(10)

(388, 22)


,Neighborhood,Arcade,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Line,Bus Station,Bus Stop,Café,...,Metro Station,Office,Outdoor Sculpture,Parking,Platform,Rental Car Location,Road,Trail,Train,Train Station
0,Adams-Normandie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adams-Normandie,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Adams-Normandie,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adams-Normandie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adams-Normandie,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Adams-Normandie,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Alhambra,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,Alhambra,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,Altadena,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Altadena,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
##Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

LA_grouped = LA_venues_onehot.groupby('Neighborhood').mean().reset_index()
LA_grouped

,Neighborhood,Arcade,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Line,Bus Station,Bus Stop,Café,...,Metro Station,Office,Outdoor Sculpture,Parking,Platform,Rental Car Location,Road,Trail,Train,Train Station
0,Adams-Normandie,0.0,0.0,0.00,0.00,0.0,0.500000,0.000000,0.166667,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
1,Alhambra,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,1.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
2,Altadena,0.0,0.0,0.50,0.50,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
3,Arcadia,0.0,0.0,0.00,0.00,0.0,1.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
4,Arlington Heights,0.0,0.0,0.00,0.00,0.0,1.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
5,Athens,0.0,0.0,0.00,0.00,0.0,1.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
6,Avalon,0.0,0.0,0.25,0.25,0.0,0.250000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.25,0.00,0.0,0.0,0.000000
7,Azusa,0.0,0.0,0.00,0.00,0.0,0.400000,0.200000,0.200000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
8,Baldwin Hills/Crenshaw,0.0,0.0,0.00,0.00,0.0,0.600000,0.200000,0.200000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000
9,Baldwin Park,0.0,0.0,0.00,0.00,0.0,0.500000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.500000


In [22]:
# Putting into a dataframe and display the top 10 venues for each neighborhood.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = LA_grouped['Neighborhood']

for ind in np.arange(LA_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Bus Line,Light Rail Station,Bus Stop,General Travel,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Station,Café
1,Alhambra,Bus Stop,Train Station,General Travel,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Line,Bus Station,Café
2,Altadena,Bike Rental / Bike Share,Bike Shop,Train Station,General Travel,Athletics & Sports,Building,Bus Line,Bus Station,Bus Stop,Café
3,Arcadia,Bus Line,Train Station,General Travel,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Station,Bus Stop,Café
4,Arlington Heights,Bus Line,Train Station,General Travel,Athletics & Sports,Bike Rental / Bike Share,Bike Shop,Building,Bus Station,Bus Stop,Café


In [24]:
# set number of clusters
kclusters = 3

LA_grouped_clustering = LA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 0, 0, 0, 1, 1, 0, 0], dtype=int32)

In [25]:
## a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [26]:


LA_merged = LA_neighs

# merge LAo_grouped with LA_data to add latitude/longitude for each neighborhood
LA_merged = LA_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

LA_merged.dropna(inplace = True) #Droppign rows with NaN

LA_merged['Cluster Labels']= LA_merged['Cluster Labels'].astype(int)

LA_merged.shape

(126, 15)

In [27]:
# create map
map_clusters = folium.Map(location=[LAlatitude, LAlongitude], zoom_start=11, width=800, height=900)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
cluster_counter=0
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_neighs['Latitude'], LA_neighs['Longitude'], LA_neighs['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    cluster_counter += 1
       
map_clusters

print(cluster_counter)

126


In [28]:
# create map
map_clusters = folium.Map(location=[LAlatitude, LAlongitude], zoom_start=11, width=800, height=900)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters